# Classes and Functions


_(c) 2022, Mark van den Brand and Lina Ochoa Venegas, Eindhoven University of Technology_

## Table of Contents

- [1. Introduction](#1.-Introduction)
- [2. The `Time` Class](#2.-The-Time-Class)
- [3. Modifiers](#3.-Modifiers)
- [4. Pure Functions](#4.-Pure-Functions)
- [5. Prototyping vs. Planning](#5.-Prototyping-vs.-Planning)
- [6. Invariants](#6.-Invariants)

## 1. Introduction

The next step is to develop functions that manipulate the programmer-defined types. We will see how these types can be passed as arguments to functions and be returned as results.
In this chapter, a functional programming style will be used.

## 2. Time

We will start with introducing another programmer-defined type: the class `Time`, together with the `__init__` and `__str__` methods.

In [ ]:
class Time:
    """
    Represents the time of day.
    attributes: hour, minute, second
    """
    
    def __init__(self, hour: int = 0, minute: int = 0, second: int = 0) -> None:
        """ 
        Initializes a Time object.
        :param hour: integer value for hour
        :param minute: integer value for minute
        :param second: integer value for second
        """
        self.hour: int = hour
        self.minute: int = minute
        self.second: int = second
            
    def __str__(self) -> str:
        """ 
        Returns a string representation of the Time object.
        :returns: string representation of the Time object.
        """
        return '{:02d}:{:02d}:{:02d}'.format(self.hour, self.minute, self.second)

We can now create `Time` objects and initialize them.

In [ ]:
time: Time = Time(11, 59, 33)
print(time)

***Functions* in contrast to *methods* are defined outside the scope of a class!**
We will develop functions that allows us to manipulate `Time` objects.
There exists two types of functions, namely **pure functions** and **modifiers**.

We will define a function to compare two `Time` objects. 
The function is defined outside the `Time` class

In [ ]:
def is_after(time1: Time, time2: Time) -> bool:
    """ 
    Returns a Boolean to indicate whether time1 is greater than time2.
    :param time1: a Time object
    :param time2: a Time object
    :returns: `True` if time1 is greater than time2, `False` otherwise.
    """
    return time1.hour > time2.hour or time1.minute > time2.minute or time1.second > time2.second


new_time: Time = Time(11, 59, 31)
is_after(time,new_time)

The previous function is defined in the **global scope** of the program, which means that the function is available to the rest of the members declared in the notebook.
Python turns symbols declared within the global scope into a module called `__main__`.
You can see that this is the name of the module storing the program execution by typing `__name__`.

In [ ]:
__name__

<div class="alert alert-success">
    <b>Do It Yourself!</b><br>
    Create the function <i>print_time_meridiem(time : Time)</i>, which takes an instance of type Time and prints the time following the 12-hour convention (e.g. 12:05 pm, 11:46 am).
</div>

In [ ]:
# Remove this line and add your code here

## 3. Modifiers

It may be useful for a function to modify the objects it gets as parameters. 
In that case, the changes are visible to the caller. 
Functions that work this way are called **modifiers**.

Hereafter, we will use the **prototype and patch** developing strategy, which is 
a way of tackling a complex problem by
starting with a simple *prototype* and *incrementally* dealing with the more complex situations.
In the next cell, a first version of the function `increment` is presented.

In [ ]:
def increment(time: Time, seconds: int) -> None:
    """ 
    Adds seconds to a Time object.
    :param time: a Time object
    :param seconds: number of seconds to be added
    :returns: a Time object increased by seconds.
    """
    time.second += seconds
    
    # fix the seconds if greater than 60
    if time.second >= 60:
        time.second -= 60
        time.minute += 1
        
    # fix the minutes if greater than 60
    if time.minute >= 60:
        time.minute -= 60
        time.hour += 1

    # fix with hours if greater than 24
    if time.hour >= 24:
        time.hour -= 24 

The function starts with adding the `seconds` to `time.second`; the remainder deals with the special cases we
saw before.

This function does not need to return the `time` object, because it is being directly modified within the function body.
Let us see an example where the object is being changed by the *modifier* function.

In [ ]:
my_time: Time = Time(20, 20, 30)
print(f'Before incrementing: {my_time}')

increment(my_time, 30)
print(f'After incrementing: {my_time}')

However, the `increment` function is not correct when the number of seconds to be added is larger than `60`.
It is then necessary to add more than 1 minute to `time.minutes`.

In [ ]:
my_time: Time = Time(20, 20, 30)
print(f'Before incrementing: {my_time}')

increment(my_time, 95)
print(f'After incrementing: {my_time}')

A straightforward solution is to use a `while` loop, but this may not be very efficient.

In [ ]:
def increment(time: Time, seconds: int) -> None:
    """ 
    Adds seconds to a Time object.
    :param time: a Time object
    :param seconds: number of seconds to be added
    :returns: a Time object increased by seconds.
    """
    time.second += seconds
    
    # fix the seconds if greater than 60
    while time.second >= 60:
        time.second -= 60
        time.minute += 1
        
    # fix the minutes if greater than 60
    while time.minute >= 60:
        time.minute -= 60
        time.hour += 1

    # fix with hours if greater than 24
    while time.hour >= 24:
        time.hour -= 24 

To improve the efficiency of our solution, we can leverage the `divmod` function, which given two numbers (i.e. *dividend* and *divisor* in the given order), returns their *quotient* and *remainder* as a tuple.

In [ ]:
(minutes, seconds) = divmod(95, 60)
print(f'minutes={minutes} seconds={seconds}')

Let us modify our solution!

In [ ]:
def increment(time: Time, seconds: int) -> None:
    """ 
    Adds seconds to a Time object.
    :param time: a Time object
    :param seconds: number of seconds to be added
    :returns: a Time object increased by seconds.
    """
    # One hour has 3600 seconds = 60 seconds x 60 minutes
    (hours, seconds) = divmod(seconds, 3600)
   
    # One minute has 60 seconds
    (minutes, seconds) = divmod(seconds, 60)
    
    time.hour += hours
    time.minute += minutes
    time.second += seconds
    
    
my_time: Time = Time(20, 20, 30)
print(f'Before incrementing: {my_time}')

increment(my_time, 3660)
print(f'After incrementing: {my_time}')

<div class="alert alert-success">
    <b>Do It Yourself!</b><br>
    Let's introduce some changes to the previous exercise. Create the modifier <i>convert_utc_modifier(time : Time, utc_units : int) -> Time</i>, which takes an instance of type Time (representing the Coordinated Universal Time -UTC) and adds the number of UTC units to it. This function adds the integer to the <i>hours</i> attribute. Remember that an hour must be equal or greater to zero, and less than 24.
</div>

In [ ]:
# Remove this line and add your code here

## 4. Pure Functions

Anything that can be done with modifiers can also be done with pure functions. 
In fact, some programming languages, also known as **functional programming languages**, 
only allow pure functions. 

There is some evidence that programs that use pure functions are faster to develop and less error-prone than programs
that use modifiers. 
It is, therefore, good practice to develop **pure functions** instead of **modifiers**.
However, modifiers are convenient at times, and functional programs tend to be less efficient.
Thus, defining a modifier from time to time is not forbidden.

The next cell shows a simple prototype of the `add_time` function.

In [ ]:
def add_time(t1: Time, t2: Time) -> Time:
    """ 
    Returns a new Time object containing the sum of two Time objects.
    :param time1: a Time object
    :param time2: a Time object
    :returns: a Time object containing the sum of two Time objects.
    """
    hour: int = t1.hour + t2.hour
    minute: int = t1.minute + t2.minute
    second: int = t1.second + t2.second
    addition: Time = Time(hour, minute, second)
    return addition

This function creates a new `Time` object (`addition`). 
Its attributes are initialized by adding
the values of the attributes of the arguments, and eventually returns the created object.

This is called a **pure function** because it does not have any side effects:
* it *does not modify any of the objects* passed to it as arguments and 
* it has no effect, like *displaying a value or getting user input*, other than returning a value.

The function of the previous cell can be tested by creating and adding 2 `Time` objects.

In [ ]:
start: Time = Time(23, 45, 0)
duration: Time = Time(1, 35, 0)
finished: Time = add_time(start, duration)

print(finished)

This is of course an illegal time.

The problem is caused by the fact that we did not keep track of the fact that there are only `60` seconds in a minute, `60` minutes in an hour, and `24` hours in a day, *or do we allow more hours in a `Time` object?*
We need to *carry* over seconds to minutes and minutes to hours.

In [ ]:
def add_time(t1: Time, t2: Time) -> Time:
    """ 
    Returns a new Time object containing the sum of two Time objects.
    :param time1: a Time object
    :param time2: a Time object
    :returns: a Time object containing the sum of two Time objects.
    """
    hour: int = t1.hour + t2.hour
    minute: int = t1.minute + t2.minute
    second: int = t1.second + t2.second
    
    # fix the seconds if greater than 60
    if second >= 60:
        second -= 60
        minute += 1

    # fix the minutes if greater than 60
    if minute >= 60:
        minute -= 60
        hour += 1
        
    # fix with hours if greater than 24
    if hour >= 24:
        hour -= 24
    
    addition: Time = Time(hour, minute, second)
    return addition


start: Time = Time(23, 45, 0)
duration: Time = Time(1, 35, 0)
finished: Time = add_time(start, duration)

print(finished)

This solution is slightly more elaborated than the previous.
But, the function is quite long and a more concise solution will be presented later in the chapter.

<div class="alert alert-success">
    <b>Do It Yourself!</b><br>
    Create the pure function <i>convert_utc(time : Time, utc_units : int) -> Time</i>, which takes an instance of type Time (representing the Coordinated Universal Time -UTC) and the number of UTC units. This function adds the integer to the <i>hours</i> attribute. Remember that an hour must be equal or greater to zero, and less than 24.
</div>

In [ ]:
# Remove this line and add your code here

## 5. Prototyping vs. Planning

The development strategy used so far in this chapter has been *prototype and patch*.
The problem with this approach is that the code becomes bulky, it may contain code to deal with a lot of corner cases.
In the `Time` case, you have to be aware of the seconds, minutes, and hours, all three may have a carry over.
This approach can be effective, especially if you do not have yet a deep understanding
of the problem, but it may involve a lot of testing.

An alternative strategy is **designed development**, in which high-level insight into the problem can
make the programming much easier. 
The more experienced you become as a programmer, the more similarities 
you see between problems. 
You start to see patterns and often these patterns can be applied to similar problems.
It often involves taking a step back and looking at the problem from a different angle. 

There are different
alternatives to represent time and maybe an alternative way of looking at time may simplify the manipulation
of time objects.
For instance, to realize that the conversion from time to seconds and back allows us just to manipulate time
in terms of the number of seconds: `1` minute is `60` seconds and `1` hour is `3600` seconds.

So, if we **convert the `Time` object to an integer value representing seconds and back, the solution might be way simpler!**

In [ ]:
def time_to_int(time: Time) -> int:
    """ 
    Converts a Time object into an integer value representing seconds.
    :param time: a Time object to be converted
    :returns: number of seconds as an Integer value.
    """
    minutes: int = 60 * time.hour + time.minute
    seconds: int = 60 * minutes + time.second
    return seconds

print(start)
print(time_to_int(start))

We need now to develop a function `int_to_time` that converts a number of seconds into hours, minutes, and seconds—that is, a `Time` object.
This function explicitly takes an integer value as argument and returns a `Time` object.
Internally, it will use the `divmod` function again.

In [ ]:
def int_to_time(seconds: int) -> Time:
    """ 
    Converts an integer value representing seconds into a Time object.
    :param seconds: as Integer value
    :returns: a Time object.
    """
    time: Time = Time()
    minutes, time.second = divmod(seconds, 60)
    time.hour, time.minute = divmod(minutes, 60)
        
    return time

**Is the function correct?**
One way to check that both functions are correct is by means of using a *consistency check*. 
In this case, we will verify with the expression `time_to_int(int_to_time(x)) == x`.

In [ ]:
x = 86300
assert time_to_int(int_to_time(x)) == x

Seems to be working! 
But, let us check some corner cases, for instance:
- `int_to_time(0)`: expecting `00:00:00`.
- `int_to_time(86400)`: expecting `00:00:00` given that 86.400 seconds is equal to 24 hours. `24:00:00` is not a legal time.

In [ ]:
my_time: Time = int_to_time(86400)
assert my_time == 1, f'Excesive number hours ({my_time.hour}) are nor being handled'

The solution presented so far does not deal with the fact that there are only `24` hours in a day.

Thus, the proposed test, `time_to_int(int_to_time(x)) == x`
does not work for values equal or greater than `86400`.

In [ ]:
def int_to_time(seconds: int) -> Time:
    """ 
    Converts an integer value representing seconds into a Time object.
    :param seconds: as Integer value
    :returns: a Time object.
    """
    time: Time = Time()
    minutes, time.second = divmod(seconds, 60)
    time.hour, time.minute = divmod(minutes, 60)
    
    # fix with hours if greater than 24
    if time.hour >= 24:
        time.hour -= 24
        
    return time

In [ ]:
my_time: Time = int_to_time(86400)
print(my_time)
assert my_time.hour == 0, f'Excesive number hours ({my_time.hour}) are not being handled'

Now, let us consider our initial check, but this time `x` will be set to `86400`.

In [ ]:
time_to_int(int_to_time(86400)) == 86400

In [ ]:
time_to_int(int_to_time(86400))

To solve this issue we can leverage the `%` operator!

In [ ]:
x = 86400
assert time_to_int(int_to_time(x)) == x % 86400

Once we are convinced that both functions are correct (according to our specifications), we can reimplement
the `add_time` and `increment` functions.

In [ ]:
def add_time(t1: Time, t2: Time) -> Time:
    """ 
    Returns a new Time object containing the sum of two Time objects.
    :param time1: a Time object
    :param time2: a Time object
    :returns: a Time object containing the sum of two Time objects.
    """
    seconds: int = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)


start: Time = Time(23, 45, 0)
duration: Time = Time(1, 35, 0)
finished: Time = add_time(start, duration)

print(finished)

In [ ]:
def increment(time: Time, seconds: int) -> None:
    """ 
    Adds seconds to a Time object.
    :param time: a Time object
    :param seconds: number of seconds to be added
    :returns: a Time object increased by seconds.
    """
    seconds: int = time_to_int(time) + seconds
    return int_to_time(seconds)


start: Time = Time(20, 20, 30)
duration: Time = 160
finished: Time = increment(start, duration)

print(finished)

Notice that the new implementation of the `increment` function is a *pure function*.
We are not directly modifying the `Time` object that we receive as paramaeter.
Instead, we are creating a new `Time` object without touching the argument, and returning this new value.

<div class="alert alert-success">
    <b>Do It Yourself!</b><br>
    These time conversion functions allow us to develop also a <i>sub_time(t1 : Time, t2 : Time) -> Time</i> function, that subtracts two time intervals. Again, make sure you that your are not 'travelling back to the future', or end up with a negative time.
</div>

In [ ]:
# Remove this line and add your code here

## 6. Invariants

A `Time` object is well-formed if the values of `minute` and `second` are between 0 and 60
(including 0 but not 60) and if the value of `hour` is between 0 and 24.
Requirements like these are called **invariants** because they should always be true. 
To put it a different way, if they are not true, something has gone wrong.

Writing code to check invariants can help detect errors and find their causes. 
The next cell contains the function `valid_time` that takes a `Time` object and returns `False` if it
violates an **invariant**.

In [ ]:
def valid_time(time: Time) -> bool:
    """ 
    Checks whether we are dealing with a correct Time object. That is,
    minutes and seconds are between [0, 60) and hours are between [0, 24).
    :param time: Time object to be checked
    :returns: `True` if the time is valid, `False` otherwise.
    """
    valid = True
    
    if time.hour < 0 or time.minute < 0 or time.second < 0:
        valid = False
    if time.hour >= 24 or time.minute >= 60 or time.second >= 60:
        valid = False
        
    return valid

At the beginning of each function you could check the arguments to make sure they are
valid.

In [ ]:
def add_time(t1: Time, t2: Time) -> Time:
    """ 
    Returns a new Time object containing the sum of two Time objects.
    :param time1: a Time object
    :param time2: a Time object
    :returns: a Time object containing the sum of two Time objects.
    """
    if not valid_time(t1) or not valid_time(t2):
        raise ValueError('Invalid Time object in add_time')
        
    seconds: int = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

Or you could use an **assert statement**, which checks a given invariant and raises an exception
if it fails.
This alternative is useful during the development process but not necessarily to inform your users about a mistake on the use of your program.

In [ ]:
def add_time(t1: Time, t2: Time) -> Time:
    """ 
    Returns a new Time object containing the sum of two Time objects.
    :param time1: a Time object
    :param time2: a Time object
    :returns: a Time object containing the sum of two Time objects.
    """
    assert valid_time(t1) and valid_time(t2), 'Invalid Time object in add_time'
    
    seconds: int = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

---

This Jupyter Notebook is based on Chapter 16 of the book Think Python.

---

# (End of Notebook)

&copy; 2022-2023 - **TU/e** - Eindhoven University of Technology